In [1]:
import os
import sys
import json
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline, BSpline
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)

with open("final_results_[3, 7, 8].json") as f:
    results = json.load(f)


In [5]:
# parameters
x_as_len = 30 # used for smoothing
n_experiments = 30

def smooth(listX, old_x_as, new_x_as): 
    # from 10 points to 100 then old is the list of size 10 and new the list of size 100
    listX = make_interp_spline(old_x_as, listX, k=2)  # type: BSpline
    listX = listX(new_x_as)
    return listX
x_new = np.linspace(0, n_experiments-1, x_as_len)
x_old = np.linspace(0, n_experiments-1, 30)



for en, runs in results.items():
    means = []
    maximums = []
    print(en, runs)
    for run, values in runs.items():
        means.append(values['means'])
        maximums.append(values['maximums'])

    meanAverage = np.mean(means, axis=0)
    stdMean = np.std(means, axis=0)
    maxAverage = np.mean(maximums, axis=0)
    stdMax =  np.std(maximums, axis=0)

    if len(meanAverage) < 30:
        break

    meanAverage = smooth(meanAverage,x_old,x_new)
    maxAverage = smooth(maxAverage,x_old,x_new)
    stdMean = smooth(stdMean,x_old,x_new)
    stdMax = smooth(stdMax,x_old,x_new)

    figure, ax = plt.subplots(figsize=(12, 6))
    
#     plt.figure(figsize=(12, 6), dpi=270)
    plt.tight_layout()    
    plt.plot(x_new, meanAverage, label='meanAverage', color='#1B2ACC')    
    plt.fill_between(np.linspace(0, n_experiments-1, x_as_len), 
                     meanAverage-stdMean, meanAverage+stdMean, 
                     alpha=.2, edgecolor='#1B2ACC', facecolor='#089FFF')

    plt.plot(x_new, maxAverage, label='maxAverage', color='#CC4F1B')
    plt.fill_between(np.linspace(0, n_experiments-1, x_as_len),
                     maxAverage-stdMax, maxAverage+stdMax,
                     alpha=.2, edgecolor='#CC4F1B', facecolor='#FF9848')
    
    
    plt.xlim(0, 29)
    plt.ylim(0, 100)

    
    # Change major ticks to show every 20.
    ax.xaxis.set_major_locator(MultipleLocator(5))
    ax.yaxis.set_major_locator(MultipleLocator(30))

    # Change minor ticks to show every 5. (20/4 = 5)
    ax.xaxis.set_minor_locator(AutoMinorLocator(5))
    ax.yaxis.set_minor_locator(AutoMinorLocator(6))

    plt.grid(which='major', color='#CCCCCC', linestyle='--')
    plt.grid(which='minor', color='#CCCCCC', linestyle=':')
    

#     plt.xticks(x_new)
#     plt.yticks(np.linspace(0, 100, 15))
    
    plt.title(f'EA2 performance enemy {en}')
    plt.ylabel('Fitness')
    plt.xlabel('Generation')
    plt.legend()
    plt.savefig(f'EA2-enemy{en}', bbox_inches='tight')
    plt.show()

[3, 7, 8] {'0': {'means': [-2.164181, 2.129633, 8.421756, 12.943743, 17.853467, 23.508591, 24.063585, 26.731227, 28.533495, 30.468605, 32.894897, 34.720965, 37.126583, 37.71416, 38.410235, 39.258459, 40.49565, 45.690925, 46.053989, 47.2398, 49.999512, 51.436748, 52.320714, 54.802884, 55.036667, 54.048284, 56.478759, 55.703292, 55.709555, 56.495269, 56.495269, 56.495269], 'maximums': [30.077788, 30.077788, 30.930545, 42.951671, 47.155079, 68.07717, 68.07717, 52.010661, 65.004405, 70.008986, 74.880806, 74.880806, 74.880806, 74.017648, 70.008986, 73.85691, 73.85691, 73.867766, 73.958514, 73.958514, 73.920901, 73.937769, 77.247206, 77.247206, 77.247206, 77.22726, 77.655253, 77.218997, 77.208795, 77.214734, 77.214734, 77.214734]}, '1': {'means': [-3.085704, 0.170544, 5.14825, 11.405567, 17.94725, 24.434754, 27.970108, 27.543686, 28.282218, 29.506489, 31.109933, 31.495787, 32.222686, 33.271442, 34.863199, 35.800037, 36.651342, 37.17337, 39.478128, 37.866305, 37.552289, 36.68938, 36.831793, 3

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [16]:
df = pd.DataFrame(results)
for i in range(1,4):
    df[str(i)] =  [np.mean(x) for x in df[str(i)].values]
df['method'] = 'EA1'
df

KeyError: '1'

In [10]:
df2 = pd.DataFrame(results_neat)
for i in range(1,4):
    df2[str(i)] =  [np.mean(x) for x in df2[str(i)].values]
df2['method'] = 'NEAT'
df2

,1,2,3,method
0,90.0,98.0,88.0,NEAT
1,90.0,94.0,78.0,NEAT
2,90.0,96.0,62.0,NEAT
3,92.0,92.0,90.0,NEAT
4,90.0,94.0,74.0,NEAT
5,90.0,92.0,76.0,NEAT
6,100.0,98.0,60.0,NEAT
7,90.0,98.0,78.0,NEAT
8,90.0,98.0,88.0,NEAT
9,92.0,96.0,86.0,NEAT


In [11]:
gain_dfs = pd.concat([df, df2])
for i in range(1, 4):
    fig = px.box(gain_dfs, x='method', y=str(i), 
                 labels={
                     str(i): "Individual Gain",
                 },
                   points="all", width=800, height=400)
    fig.update_layout(title_text=f'Enemy {i}', title_x=0.5)
    fig.show()